In [5]:
import weaviate
from weaviate.classes.init import Auth
import json

In [ ]:
from langchain.callbacks.base import BaseCallbackHandler

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(token, end="", flush=True)

In [10]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4.1",  # or your deployment
    api_version="2025-01-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    streaming=True,
    callbacks=[MyCustomHandler()],
    azure_endpoint="https://vishn-mbusn60p-swedencentral.openai.azure.com/",
    api_key="4NX3FZfJZqlhxBSkfzITV5vLqhdxvg4gz0uQJwjtxlrkU1WKkUcZJQQJ99BFACfhMk5XJ3w3AAAAACOGZLMY",
)

In [7]:
def search_documents(query):
    """
    Queries Weaviate for similar text chunks based on a natural language query.

    Args:
        query_text (str): The query text to search for.

    Returns:
        str: JSON string of context data with 'text' and 'metadata'.
    """
    # Define Weaviate Cloud connection parameters

    weaviate_url = '3tovdeihtdsygyyan0z3aa.c0.asia-southeast1.gcp.weaviate.cloud'
    weaviate_api_key = "R0hodm5yeHlmeTdmMU1DU19Yc2ErR0dmNXA1djZLVWFpc2hMaEdqbDBNbjRWVUVhVng4ZVJ3QzZxbWtVPV92MjAw"

    context_data = []
    
    # Connect to Weaviate Cloud
    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key)
    )
    if not client.is_ready():
        raise RuntimeError("Weaviate client not ready!")
    
    # Get collection
    collection = client.collections.get("PdfChunks")

    # Query for similar text chunks
    response = collection.query.near_text(
        query=query,  # The model provider integration will automatically vectorize the query
        limit=10
    )
    client.close()
    
    for obj in response.objects:
        context_data.append(obj.properties)

    return json.dumps(context_data)

In [11]:
from langchain.agents import initialize_agent, Tool, AgentType

tools = [
    Tool(
        name="search_documents",
        func=search_documents,
        description="Searches Weaviate for similar text chunks based on a natural language query. Returns JSON string of context data with 'text' and 'metadata'."
    )
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=False,
)


In [16]:
response = agent.invoke("What's the role of ceo in contoso?")


The role of the CEO (Chief Executive Officer) at Contoso Electronics is to provide strategic direction and oversight to ensure the company’s long-term success and profitability. Here are the key responsibilities of the CEO at Contoso:

- Develop and implement strategy and objectives to ensure the financial success and long-term growth of the company.
- Provide strategic guidance and direction to the executive team and manage the day-to-day operations of the company.
- Ensure compliance with all laws and regulations.
- Oversee the development of marketing and advertising strategies.
- Develop and manage relationships with key stakeholders.
- Represent the company in speaking engagements and other public events.
- Manage and negotiate contracts with vendors and suppliers.
- Monitor industry trends and identify opportunities for growth.
- Develop and maintain a positive company culture.

Qualifications typically include a bachelor’s degree in business or a related field, 10+ years of expe